In [ ]:
from glob import glob
from pathlib import Path
import cv2
import albumentations as A

In [ ]:
'''
This notebook is used to test the augmentation pipeline for binary segmentation.
https://ai/docs/getting_started/image_augmentation/
Compose:
- piecewise affine (p=0.9)
- one of (probability=0.8): (not on mask)
    - randomgamma (probability=0.25)
    - randombrightnesscontrast (probability=0.25)
    - randomtonecurve (probability=0.25)
    - clahe (probability=0.25)
- one of (probability=0.75):
    - verticalflip (probability=0.5)
    - randomrotate90 (probability=0.5)
'''

transform = A.Compose([
	A.PiecewiseAffine(p=0.9),
	A.OneOf([A.RandomGamma(p=0.25), A.RandomBrightnessContrast(p=0.25), A.RandomToneCurve(p=0.25), A.CLAHE(p=0.25)], 0.8),
	A.OneOf([A.VerticalFlip(p=0.5), A.RandomRotate90(p=0.5)], 0.75)
])

In [ ]:
# Change this to match your data paths
images = [cv2.imread(f, 0) for f in glob('no_skeleton/train/img/*.png')]
masks = [cv2.imread(f, 0) for f in glob('no_skeleton/train/mask/*.png')]

In [ ]:
Path('transforms_output').mkdir(parents=True, exist_ok=True)

In [ ]:
for i in range(25):
    transformed = transform(image=images[0], mask=masks[0])
    cv2.imwrite(f'transforms_output/{i}_img.png', transformed['image'])
    cv2.imwrite(f'transforms_output/{i}_mask.png', transformed['mask'])